In [1]:
import spacy
from spacy.training import Example
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/kaggle/input/xddddd-yasha/anti-punto-switcher-public.csv')
test_data = pd.read_csv('/kaggle/input/xddddd-yasha/anti-punto-switcher-private.csv')

In [2]:
def create_training_data(df):
    training_data = []
    for text, label in zip(df['text'], df['label']):
        text_words = text.split()
        label_words = label.split()
        
        entities = []
        offset = 0
        
        for t_word, l_word in zip(text_words, label_words):
            start = text.find(t_word, offset)
            end = start + len(t_word)
            
            if t_word != l_word:
                entities.append((start, end, "INCORRECT"))
            else:
                entities.append((start, end, "CORRECT"))
            offset = end

        training_data.append((text, {"entities": entities}))
    
    return training_data

train_data = create_training_data(data)

In [4]:
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 82.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.

In [5]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example

In [6]:
nlp = spacy.blank("ru")
db = DocBin()

for text, annotations in train_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annotations['entities']:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print(f"Skipping entity ({start}, {end}, {label}) in sentence: {text}")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./train.spacy")

In [17]:
import os

config_content = """
[paths]
train = "/kaggle/working/train.spacy"
dev = "/kaggle/working/train.spacy"

[system]
gpu_allocator = "pytorch"

[nlp]
lang = "ru"
pipeline = ["transformer","ner"]
batch_size = 128

[components]

[components.transformer]
factory = "transformer"

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v3"
name = "bert-base-multilingual-uncased"
tokenizer_config = {"use_fast": true}

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.ner.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = 52
gpu_allocator = "pytorch"

[training.optimizer]
@optimizers = "Adam.v1"

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 5e-5

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256
"""

with open("/kaggle/working/config.cfg", "w") as f:
    f.write(config_content)

In [20]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id=0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2024-08-18 12:04:10,958] [INFO] Set up nlp object from config
[2024-08-18 12:04:10,998] [INFO] Pipeline: ['transformer', 'ner']
[2024-08-18 12:04:11,004] [INFO] Created vocabulary
[2024-08-18 12:04:11,004] [INFO] Finished initializing nlp object
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/cupy/cuda/compiler.py:233: PerformanceWarning: Jitify is performing a one-time only warm-up to populate the persistent cache, this may take a few seconds and will be improved in a future release...
  jitify._init_module()
[2024-08-18 12:05:42,708] [INFO] Initialized pipeline components: ['transfor

In [ ]:
import spacy

nlp = spacy.load("./output/model-best")

test_text = "и старательности полкового rjvfylbhf был"
doc = nlp(test_text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
def switch_layout(word):
    layout_mapping = str.maketrans("qwertyuiop[]asdfghjkl;'zxcvbnm,./", "йцукенгшщзхъфывапролджэячсмитьбюё")
    return word.translate(layout_mapping)

def correct_text(text):
    doc = nlp(text)
    corrected_text = []
    for token in doc:
        if token.ent_type_ == "INCORRECT":
            corrected_text.append(switch_layout(token.text))
        else:
            corrected_text.append(token.text)
    return " ".join(corrected_text)

results = []
for text in test_data['text']:
    results.append(correct_text(text))

with open('output.csv', 'w') as f:
    for line in results:
        f.write(line + '\n')